In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

In [2]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_B5():
    data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv'
    urllib.request.urlretrieve(data_url, 'daily-max-temperatures.csv')

    time_step = []
    temps = []

    with open('daily-max-temperatures.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        step = 0
        for row in reader:
            temps.append(float(row[1])) # YOUR CODE HERE
            time_step.append(step) # YOUR CODE HERE
            step=step + 1

    series= np.array(temps) # YOUR CODE HERE

    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
    time=np.array(time_step)

    # DO NOT CHANGE THIS CODE
    split_time=2500

    time_train = time[:split_time] # YOUR CODE HERE
    x_train = series[:split_time] # YOUR CODE HERE
    time_valid = time[split_time:] # YOUR CODE HERE
    x_valid = series[split_time:] # YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    window_size=64
    batch_size=256
    shuffle_buffer_size=1000

    train_set=windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)

    model=tf.keras.models.Sequential([
        # YOUR CODE HERE.
        tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding="causal", activation="relu", input_shape=[None, 1]),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # YOUR CODE HERE
    model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(), metrics=["mae"])

    model.fit(train_set, epochs=20)

    return model

In [3]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_B5()
    model.save("model_B5.h5")

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)
Epoch 1/20
10/10 [==============================] - 4s 160ms/step - loss: 0.0486 - mae: 0.1764
Epoch 2/20
10/10 [==============================] - 1s 106ms/step - loss: 0.0146 - mae: 0.0870
Epoch 3/20
10/10 [==============================] - 1s 109ms/step - loss: 0.0131 - mae: 0.0922
Epoch 4/20
10/10 [==============================] - 1s 108ms/step - loss: 0.0129 - mae: 0.0856
Epoch 5/20
10/10 [==============================] - 1s 112ms/step - loss: 0.0122 - mae: 0.0847
Epoch 6/20
10/10 [==============================] - 1s 108ms/step - loss: 0.0119 - mae: 0.0842
Epoch 7/20
10/10 [==============================] - 1s 111ms/step - loss: 0.0117 - mae: 0.0831
Epoch 8/20
10/10 [==============================] - 1s 110ms/step - loss: 0.0116 - mae: 0.0826
Epoch 9/20
10/10 [==============================] - 1s 112ms/step - los

c:\Users\ilham\miniconda3\envs\vision1\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
